<a href="https://colab.research.google.com/github/HausReport/ClubRaiders/blob/master/notebooks/History.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys

!{sys.executable} -m pip install --force-reinstall --upgrade "git+https://github.com/HausReport/ClubRaiders.git#egg=craid"
import craid.eddb.loader.DataProducer as dp
import pandas as pd
import logging
import plotly.graph_objs as go
from numpy import *


logging.getLogger().addHandler(logging.StreamHandler())
logging.getLogger().level = logging.DEBUG

arrays = dp.getDataArrays()
df: pd.DataFrame = arrays['dataFrame']

  Cloning https://github.com/HausReport/ClubRaiders.git to /tmp/pip-install-47ublp1f/craid
  Running command git clone -q https://github.com/HausReport/ClubRaiders.git /tmp/pip-install-47ublp1f/craid
     |████████████████████████████████| 71kB 1.9MB/s 
     |████████████████████████████████| 3.4MB 7.0MB/s 
     |████████████████████████████████| 194kB 31.2MB/s 
     |████████████████████████████████| 1.8MB 32.4MB/s 
     |████████████████████████████████| 1.2MB 45.5MB/s 
     |████████████████████████████████| 102kB 11.7MB/s 
     |████████████████████████████████| 10.1MB 48.2MB/s 
     |████████████████████████████████| 71kB 9.6MB/s 
     |████████████████████████████████| 583kB 48.1MB/s 
     |████████████████████████████████| 14.6MB 182kB/s 
     |████████████████████████████████| 235kB 44.6MB/s 
     |████████████████████████████████| 184kB 44.6MB/s 
     |████████████████████████████████| 133kB 52.4MB/s 
     |████████████████████████████████| 450kB 48.7MB/s 
     |██████████████

Loading from AWS
1- downloading to: /tmp/smol-factions.jsonl.gz
2 - downloading [https://erlaed.s3.us-east-2.amazonaws.com/smol-factions.jsonl.gz] to [/tmp/smol-factions.jsonl.gz] data file.
Starting new HTTPS connection (1): erlaed.s3.us-east-2.amazonaws.com:443
https://erlaed.s3.us-east-2.amazonaws.com:443 "GET /smol-factions.jsonl.gz HTTP/1.1" 200 15629
Found data file: /tmp/smol-factions.jsonl.gz
Read 671 lines of faction data
1- downloading to: /tmp/smol-systems_populated.jsonl.gz


0:197,246,976
0.5:198,991,872


2 - downloading [https://erlaed.s3.us-east-2.amazonaws.com/smol-systems_populated.jsonl.gz] to [/tmp/smol-systems_populated.jsonl.gz] data file.
Starting new HTTPS connection (1): erlaed.s3.us-east-2.amazonaws.com:443
https://erlaed.s3.us-east-2.amazonaws.com:443 "GET /smol-systems_populated.jsonl.gz HTTP/1.1" 200 26175
Found data file: /tmp/smol-systems_populated.jsonl.gz
Read 174 lines of systems data
Populated club system keys
Club system lookup set has {174} items
Populated faction instances: {1047} items.
1- downloading to: /tmp/smol-stations.jsonl.gz
2 - downloading [https://erlaed.s3.us-east-2.amazonaws.com/smol-stations.jsonl.gz] to [/tmp/smol-stations.jsonl.gz] data file.
Starting new HTTPS connection (1): erlaed.s3.us-east-2.amazonaws.com:443


1:201,379,840
2:201,920,512


https://erlaed.s3.us-east-2.amazonaws.com:443 "GET /smol-stations.jsonl.gz HTTP/1.1" 200 125037
Found data file: /tmp/smol-stations.jsonl.gz
Read 675 lines of station data, adding 675

station controlling faction nexistWFM-77Z
station controlling faction nexistWFM-77Z
station controlling faction nexistV3V-73Z
station controlling faction nexistV3V-73Z
station controlling faction nexistKFQ-79L
station controlling faction nexistKFQ-79L
station controlling faction nexistQ5T-64V
station controlling faction nexistQ5T-64V
station controlling faction nexistJZF-33G
station controlling faction nexistJZF-33G
station controlling faction nexistXNK-TVV
station controlling faction nexistXNK-TVV
station controlling faction nexistQ7N-GKZ
station controlling faction nexistQ7N-GKZ
station controlling faction nexistX7M-B6Q
station controlling faction nexistX7M-B6Q
station controlling faction nexistJBL-3XQ
station controlling faction nexistJBL-3XQ
station controlling faction nexistV1Q-G1X
station controlli

3:202,145,792


station controlling faction nexistX6Z-02G
station controlling faction nexistL9Y-B8Z
station controlling faction nexistL9Y-B8Z
station controlling faction nexistKHF-N0Y
station controlling faction nexistKHF-N0Y
station controlling faction nexistX6Z-02G
station controlling faction nexistX6Z-02G
station controlling faction nexistL9Y-B8Z
station controlling faction nexistL9Y-B8Z
station controlling faction nexistKHF-N0Y
station controlling faction nexistKHF-N0Y
station controlling faction nexistThunderhead Garrison
station controlling faction nexistThunderhead Garrison
station controlling faction nexistLFF-NQZ
station controlling faction nexistLFF-NQZ
station controlling faction nexistJZB-G3L
station controlling faction nexistJZB-G3L
station controlling faction nexistK1X-51H
station controlling faction nexistK1X-51H
station controlling faction nexistQ1T-T4K
station controlling faction nexistQ1T-T4K
station controlling faction nexistX9V-GQZ
station controlling faction nexistX9V-GQZ
station 

In [ ]:
df.loc[df['control'] == True, 'marker_size'] = 8           # Medium is not home/control
df.loc[df['control'] == False, 'marker_size'] = 5          # Small is not home/not control
df.loc[df['isHomeSystem'] == True, 'marker_size'] = 15     # Large is home
df.loc[df['control'] == True, 'color'] = "#ffbf00"         # Yellow is control/not home
df.loc[df['control'] == False, 'color'] = "#00ff00"        # Green is not home/not control
df.loc[df['isHomeSystem'] == True, 'color'] = "#ff0000"    # Red is homesystem
#df['htext'] = df[['systemName', 'factionName', 'influence', 'isHomeSystem','control','difficulty']].astype(str).agg(':'.join, axis=1)
df['htext'] = df[['systemName', 'factionName']].agg('\n'.join, axis=1)

In [ ]:
from craid.eddb.Aware import Aware
Aware.getSystemById(292).getPower()

'Li Yong-Rui'